#### Configure mlrun project:

In [21]:
%config Completer.use_jedi = False

import mlrun
from mlrun import get_or_create_project

image = "mlrun/mlrun"
project_name = "langchain-example"
project = get_or_create_project(project_name, context="./", allow_cross_project=True)

> 2025-12-03 07:17:36,530 [info] Project loaded successfully: {"project_name":"langchain-example-10"}


#### Temp private marketplace:

In [22]:
import mlrun.common.schemas

# Add a custom hub to the top of the list
private_source = mlrun.common.schemas.IndexedHubSource(
    index=1,
    source=mlrun.common.schemas.HubSource(
        metadata=mlrun.common.schemas.HubObjectMetadata(
            name="hub15", description="a private hub"
        ),
        spec=mlrun.common.schemas.HubSourceSpec(
            path="https://raw.githubusercontent.com/royischoss/marketplace/refs/heads/master", 
            channel="master",
        ),
    ),
)
db = mlrun.get_run_db()
db.create_hub_source(private_source)

IndexedHubSource(index=1, source=HubSource(kind=HubSource, metadata=HubObjectMetadata(name='hub15', description='a private hub', labels={}, updated=datetime.datetime(2025, 12, 3, 7, 17, 54, 932778, tzinfo=datetime.timezone.utc), created=datetime.datetime(2025, 12, 3, 7, 17, 54, 932778, tzinfo=datetime.timezone.utc)), spec=HubSourceSpec(path='https://raw.githubusercontent.com/royischoss/marketplace/refs/heads/master', channel='master', credentials={}), status=ObjectStatus(state='created')))

#### Create openai secret:

In [23]:
# Create project secrets for project
secrets = {"OPENAI_API_KEY": "eyJhbGciOiJSUzI1NiIsInR5cCIgOiAiSldUIiwia2lkIiA6ICJhZXNKN2kxNGNidnVuTU40MTJrOU5yZ2ROeENhTlJudTNPbC1TU08ycFlJIn0.eyJleHAiOjE3NjQ3NDc3MDIsImlhdCI6MTc2NDc0NTkwMywiYXV0aF90aW1lIjoxNzY0NzQ1ODgzLCJqdGkiOiI5MjZiZWQ1ZS0wYWI1LTQyZmQtYjVlNi0wM2M0M2Q1ZTNkNWEiLCJpc3MiOiJodHRwczovL2F1dGgubWNraW5zZXkuaWQvYXV0aC9yZWFsbXMvciIsImF1ZCI6ImJjZDIzNzI4LTNkMjctNDQ3Yy1hMGE5LWVhY2FmMzkzYTZmNSIsInN1YiI6IjFiNGVlOThiLWRmYjUtNDBmZS05ODcwLWRjNWVlNTk2ZDM1MSIsInR5cCI6IklEIiwiYXpwIjoiYmNkMjM3MjgtM2QyNy00NDdjLWEwYTktZWFjYWYzOTNhNmY1Iiwic2Vzc2lvbl9zdGF0ZSI6IjI1ZmI5YTk3LTAzOWYtNDRhMi1hMDM0LWNlYzcwOTlkMWZhMCIsImF0X2hhc2giOiIwb1plcldMTXBPMXdpM1NNS2Y1M0tBIiwibmFtZSI6IlJveSBTY2hvc3NiZXJnZXIiLCJnaXZlbl9uYW1lIjoiUm95IiwiZmFtaWx5X25hbWUiOiJTY2hvc3NiZXJnZXIiLCJwcmVmZXJyZWRfdXNlcm5hbWUiOiIxZjgwM2Y4N2YxNDkxZjQxIiwiZW1haWwiOiJSb3lfU2Nob3NzYmVyZ2VyQG1ja2luc2V5LmNvbSIsImFjciI6IjEiLCJzaWQiOiIyNWZiOWE5Ny0wMzlmLTQ0YTItYTAzNC1jZWM3MDk5ZDFmYTAiLCJlbWFpbF92ZXJpZmllZCI6dHJ1ZSwiZm1ubyI6IjMzODc0MCIsImdyb3VwcyI6WyI2NGE4NjBmOC03YjJiLTRmOWMtYmYzOS1iZjZjODJmZWM2NTciLCJBbGwgRmlybSBVc2VycyJdfQ.W_tq6nAMi_HWPp0v7G5Is52RguG6uq2XvlEMae2MoogQAY_6rnDcD3suPnszEKSOSTf7Lns3XJepcaWRFfimjoN1vexkJtL8nXfsIgZAUqicqrbIUpbTSU1lycGkuX4UcLphFPcSkrpiEqX1-7RsAZN5cmQPyYmW11zJeJjd6ZWq934HvZE6431j_d3md9v2K1OQu8homFEOkepu7BfLxXylqcs3k_1fehGiSCgcn5IXpL6TdK75Orkahadc1CaoW1jfzyDe6E2aTWMJ56KWuw0LDKLWsLLnBA5j7nZSXRKixMGCXz-NdXpEbz_0wxS1Me4U9JGQt22yzoCPqQmiuw"}
project.set_secrets(secrets=secrets, provider="kubernetes")

#### Write your python file:

In [ ]:
%%writefile wximq.py

# Langchain impoets for Agent initialization and use:
from langchain_openai import ChatOpenAI
from langchain.agents import AgentExecutor, create_react_agent
from langchain_core.tools import tool
from langchain_core.prompts import PromptTemplate

# mlrun imports:
import mlrun
from mlrun.serving import Model

# General imports:
from typing import Any
import os


@tool
def calculator(expression: str) -> str:
    """
    Use this tool to evaluate a mathematical expression.
    It can handle addition, mutliplication, subtraction, division and exponents.
    Example: `calculator("2 + 2")` or `calculator('3**4')`
    """
    try:
        return str(eval(expression))
    except Exception as e:
        return f"Error evaluating expression: {e}"



class ModuleModelWrapper(Model):
    
    def __init__(
            self,
            *args,
            prompt_template:str,
            **kwargs
    ) -> None:
        self.prompt_template = prompt_template
        print(f"[Roy] {self.prompt_template}, {ModuleModelWrapper._dict_fields}")
        super().__init__(**kwargs)
        self.executor = None
        

    def predict(self, body: Any, **kwargs) -> Any:
        if not self.executor:
            raise RuntimeError("Model not loaded. Call load() before predict().")
        answer = self.executor.invoke(
            {
            "input": body.pop("question", ""),
            # Custom fields if your agent template uses them
            },
            config={
                "max_iterations": 8,
                "callbacks": [],
                "run_name": "my_run",
                "metadata": {"request_id": "123"},
                }
            )
        body = answer
        return answer
        

    def load(self):
        if not self.executor:
            os.environ["OPENAI_API_KEY"] = mlrun.get_secret_or_env("OPENAI_API_KEY")
            model = ChatOpenAI(model="gpt-4o-mini", openai_api_key=os.environ["OPENAI_API_KEY"],
                               openai_api_base="https://openai.prod.ai-gateway.quantumblack.com/64a860f8-7b2b-4f9c-bf39-bf6c82fec657/v1"
                              )
            agent = create_react_agent(llm=model, tools=[calculator], prompt=PromptTemplate.from_template(self.prompt_template))
            self.executor = AgentExecutor(
                agent=agent,
                tools=[calculator],
                
            )
            

#### Import the module from the hub:

In [81]:
module = mlrun.import_module("hub://hub15/agent_wrapper")

agent = module.AgentDeployer(
            project_name=project_name,
            agent_name="langchain_agent",
            model_class_name="ModuleModelWrapper",
            function="langchain_model.py",
            model_params={"prompt_template": """
            Answer the following questions as best you can.
            You have access to the following tools:
            {tools}
            Use the following format:
            Question: the input question you must answer
            Thought: you should always think about what to do
            Action: the action to take, should be one of [{tool_names}]
            Action Input: the input to the action
            Observation: the result of the action
            ... (this Thought/Action/Action Input/Observation can repeat N times)
            Thought: I now know the final answer
            Final Answer: the final answer to the original input question
            
            Begin!
            Question: {input}
            Thought:{agent_scratchpad}
            """},
            result_path="output",
            inputs_path="input",
            requirements=["langchain==0.3.7", "langchain-openai==0.2.5", "langchain-community==0.3.3"],
            image = image,
)

> 2025-12-03 10:55:46,194 [info] Project loaded successfully: {"project_name":"langchain-example-10"}
> 2025-12-03 10:55:46,463 [info] Model monitoring credentials were set successfully. Please keep in mind that if you already had model monitoring functions / model monitoring infra / tracked model server deployed on your project, you will need to redeploy them. For redeploying the model monitoring infra, first disable it using `project.disable_model_monitoring()` and then enable it using `project.enable_model_monitoring()`.
details: MLRunConflictError("The following model-montioring infrastructure functions are already deployed, aborting: ['model-monitoring-controller', 'model-monitoring-writer']\nIf you want to redeploy the model-monitoring controller (maybe with different base-period), use update_model_monitoring_controller.If you want to redeploy all of model-monitoring infrastructure, call disable_model_monitoringbefore calling enable_model_monitoring again.")


In [82]:
func = agent.get_function()

In [83]:
func.deploy()

> 2025-12-03 10:55:51,204 [info] Starting remote function deploy
2025-12-03 10:55:51  (info) Deploying function
2025-12-03 10:55:51  (info) Building
2025-12-03 10:55:51  (info) Staging files and preparing base images
2025-12-03 10:55:51  (warn) Using user provided base image, runtime interpreter version is provided by the base image
2025-12-03 10:55:51  (info) Building processor image
2025-12-03 10:57:17  (info) Build complete
2025-12-03 10:57:25  (info) Function deploy complete
> 2025-12-03 10:57:32,590 [info] Model endpoint creation task completed with state succeeded
> 2025-12-03 10:57:32,590 [info] Successfully deployed function: {"external_invocation_urls":["langchain-example-10-langchain-agent-serving-function.default-tenant.app.vmdev75.lab.iguazeng.com/"],"internal_invocation_urls":["nuclio-langchain-example-10-langchain-agent-serving-function.default-tenant.svc.cluster.local:8080"]}


'http://langchain-example-10-langchain-agent-serving-function.default-tenant.app.vmdev75.lab.iguazeng.com/'

In [84]:
func.invoke("./", {"question" : "If a pizza costs $18.75 and I want to buy 3, plus a 15% tip, what is the total cost?"})

{'input': 'If a pizza costs $18.75 and I want to buy 3, plus a 15% tip, what is the total cost?',
 'output': '$64.69'}